## Agents

Agents use an LLM to determine which actions to perform and in what order. An action can be either using a tool and observing its output or returning it to the user. To use an agent, in addition to the concept of an LLM, it is important to understand a new concept and that of a "tool".

### Tools

Tools are functions that agents can use to interact with the world. These tools can be common utilities (e.g. search), other chains, or even other agents.

In [ ]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

In [ ]:
from langchain.agents import load_tools
from langchain_openai import OpenAI
from langchain.agents import AgentType

llm=OpenAI()

tool_names = ["llm-math"]
tools = load_tools(tool_names, llm=llm)
tools

In [ ]:
from langchain.agents import initialize_agent

agent = initialize_agent(tools,
                         llm,
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                         verbose=True,
                         max_iterations=3)

Lets have a look at the ReAct template

In [ ]:
agent.run("How many members does the A Team have?")

In [ ]:
agent.run("What is 100 devided by 25?")

### Custom Tools

You can also create your own tools by creating a class that inherits from BaseTool.

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores.faiss import FAISS

embeddings = OpenAIEmbeddings()

vectorstore = FAISS.load_local("index", embeddings)

In [ ]:
from typing import Optional
from langchain.tools import BaseTool
from langchain.callbacks.manager import AsyncCallbackManagerForToolRun, CallbackManagerForToolRun

class CustomSearchTool(BaseTool):
    name = "restaurant search"
    description = "useful for when you need to answer questions about our restaurant"

    def _run(self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None) -> str:
        store = vectorstore.as_retriever()
        docs = store.get_relevant_documents(query)
        text_list = [doc.page_content for doc in docs]
        return "\n".join(text_list)

    async def _arun(self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("custom_search does not support async")

In [ ]:
from langchain.agents import AgentType

tools = [CustomSearchTool()]
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True, max_iterations=3)

In [ ]:
agent.run("Who are the members of the A-Team?")

In [ ]:
agent.run("When does the restaurant open?")

### Chat Agents

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain_openai import ChatOpenAI

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
llm = ChatOpenAI()
agent_chain = initialize_agent(tools, llm, agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION, verbose=True, memory=memory)

In [ ]:
print(agent_chain.agent.llm_chain.prompt.messages)

In [ ]:
agent_chain.run(input="What is the capital of France?")

In [ ]:
agent_chain.run(input="Any suggestions what to visit there?")